# Load the main data sheets:

In [9]:
import pandas as pd

# File paths
data_file = r"C:\Users\bezis\Downloads\ethiopia-fi-forecast\data\raw\ethiopia_fi_unified_data.xlsx"
reference_file = r"C:\Users\bezis\Downloads\ethiopia-fi-forecast\data\raw\reference_codes.xlsx"

# Load sheets
data = pd.read_excel(data_file, sheet_name='ethiopia_fi_unified_data')
impact_links = pd.read_excel(data_file, sheet_name='Impact_sheet')
reference_codes = pd.read_excel(reference_file, sheet_name=0)

# Quick preview
print(data.head())
print(impact_links.head())
print(reference_codes.head())


  record_id  record_type category  pillar               indicator  \
0  REC_0001  observation      NaN  ACCESS  Account Ownership Rate   
1  REC_0002  observation      NaN  ACCESS  Account Ownership Rate   
2  REC_0003  observation      NaN  ACCESS  Account Ownership Rate   
3  REC_0004  observation      NaN  ACCESS  Account Ownership Rate   
4  REC_0005  observation      NaN  ACCESS  Account Ownership Rate   

  indicator_code indicator_direction  value_numeric value_text  value_type  \
0  ACC_OWNERSHIP       higher_better           22.0        NaN  percentage   
1  ACC_OWNERSHIP       higher_better           35.0        NaN  percentage   
2  ACC_OWNERSHIP       higher_better           46.0        NaN  percentage   
3  ACC_OWNERSHIP       higher_better           56.0        NaN  percentage   
4  ACC_OWNERSHIP       higher_better           36.0        NaN  percentage   

   ... impact_direction impact_magnitude impact_estimate lag_months  \
0  ...              NaN              NaN     

- 1a: Count records by type, pillar, source_type, confidence

In [10]:
# Counts by record_type
print("Record Type Counts:\n", data['record_type'].value_counts())

# Counts by pillar
print("\nPillar Counts:\n", data['pillar'].value_counts())

# Counts by source_type
print("\nSource Type Counts:\n", data['source_type'].value_counts())

# Counts by confidence
print("\nConfidence Counts:\n", data['confidence'].value_counts())


Record Type Counts:
 record_type
observation    30
event          10
target          3
Name: count, dtype: int64

Pillar Counts:
 pillar
ACCESS           16
USAGE            11
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

Source Type Counts:
 source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

Confidence Counts:
 confidence
high      40
medium     3
Name: count, dtype: int64


- 1b: Temporal range of observations

In [11]:
# Make sure date column is datetime
data['observation_date'] = pd.to_datetime(data['observation_date'], errors='coerce')

# Find min and max dates
obs = data[data['record_type']=='observation']
print("Observation date range:", obs['observation_date'].min(), "to", obs['observation_date'].max())


Observation date range: 2014-12-31 00:00:00 to 2025-12-31 00:00:00


- 1c: List unique indicators and coverage

In [12]:
indicators = obs['indicator_code'].unique()
print("Indicators:", indicators)

# Coverage count
coverage = obs.groupby('indicator_code')['value_numeric'].count()
print("\nIndicator Coverage:\n", coverage)


Indicators: ['ACC_OWNERSHIP' 'ACC_MM_ACCOUNT' 'ACC_4G_COV' 'ACC_MOBILE_PEN'
 'ACC_FAYDA' 'USG_P2P_COUNT' 'USG_P2P_VALUE' 'USG_ATM_COUNT'
 'USG_ATM_VALUE' 'USG_CROSSOVER' 'USG_TELEBIRR_USERS' 'USG_TELEBIRR_VALUE'
 'USG_MPESA_USERS' 'USG_MPESA_ACTIVE' 'USG_ACTIVE_RATE' 'AFF_DATA_INCOME'
 'GEN_GAP_ACC' 'GEN_MM_SHARE' 'GEN_GAP_MOBILE']

Indicator Coverage:
 indicator_code
ACC_4G_COV            2
ACC_FAYDA             3
ACC_MM_ACCOUNT        2
ACC_MOBILE_PEN        1
ACC_OWNERSHIP         6
AFF_DATA_INCOME       1
GEN_GAP_ACC           2
GEN_GAP_MOBILE        1
GEN_MM_SHARE          1
USG_ACTIVE_RATE       1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_CROSSOVER         1
USG_MPESA_ACTIVE      1
USG_MPESA_USERS       1
USG_P2P_COUNT         2
USG_P2P_VALUE         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
Name: value_numeric, dtype: int64


- 1d: List events and their dates

In [13]:
events = data[data['record_type']=='event']
print(events[['record_id','category','pillar','observation_date']])


   record_id        category pillar observation_date
33  EVT_0001  product_launch    NaN       2021-05-17
34  EVT_0002    market_entry    NaN       2022-08-01
35  EVT_0003  product_launch    NaN       2023-08-01
36  EVT_0004  infrastructure    NaN       2024-01-01
37  EVT_0005          policy    NaN       2024-07-29
38  EVT_0006       milestone    NaN       2024-10-01
39  EVT_0007     partnership    NaN       2025-10-27
40  EVT_0008  infrastructure    NaN       2025-12-18
41  EVT_0009          policy    NaN       2021-09-01
42  EVT_0010         pricing    NaN       2025-12-15


- 1e: Review impact_links

In [14]:
print(impact_links[['record_id','parent_id','pillar','indicator','impact_direction','impact_magnitude','lag_months']])


   record_id parent_id         pillar  \
0   IMP_0001  EVT_0001         ACCESS   
1   IMP_0002  EVT_0001          USAGE   
2   IMP_0003  EVT_0001          USAGE   
3   IMP_0004  EVT_0002         ACCESS   
4   IMP_0005  EVT_0002  AFFORDABILITY   
5   IMP_0006  EVT_0003          USAGE   
6   IMP_0007  EVT_0003         ACCESS   
7   IMP_0008  EVT_0004         ACCESS   
8   IMP_0009  EVT_0004         GENDER   
9   IMP_0010  EVT_0005  AFFORDABILITY   
10  IMP_0011  EVT_0007          USAGE   
11  IMP_0012  EVT_0007          USAGE   
12  IMP_0013  EVT_0008          USAGE   
13  IMP_0014  EVT_0010  AFFORDABILITY   

                                            indicator impact_direction  \
0                Telebirr effect on Account Ownership         increase   
1                   Telebirr effect on Telebirr Users         increase   
2                 Telebirr effect on P2P Transactions         increase   
3                     Safaricom effect on 4G Coverage         increase   
4             

- Structure your additions

In [17]:
import pandas as pd



# Use the full path
data = pd.read_excel(r"C:\Users\bezis\Downloads\ethiopia-fi-forecast\data\raw\ethiopia_fi_unified_data.xlsx",
                     sheet_name='ethiopia_fi_unified_data')

impact_links = pd.read_excel(r"C:\Users\bezis\Downloads\ethiopia-fi-forecast\data\raw\ethiopia_fi_unified_data.xlsx",
                             sheet_name='Impact_sheet')
# Example: Adding a new observation
new_observation = {
    "record_id": "REC_0031",
    "record_type": "observation",
    "category": None,
    "pillar": "ACCESS",
    "indicator": "Account Ownership Rate",
    "indicator_code": "ACC_OWNERSHIP",
    "indicator_direction": "higher_better",
    "value_numeric": 60,
    "value_text": None,
    "value_type": "percentage",
    "observation_date": "2026-12-31",
    "source_name": "World Bank Findex",
    "source_url": "https://globalfindex.worldbank.org/",
    "confidence": "high",
    "collected_by": "Esunma",
    "collection_date": "2026-01-31",
    "original_text": "60% of adults have accounts",
    "notes": "Latest Findex data for Ethiopia",
}

# Convert to DataFrame and append
data = pd.concat([data, pd.DataFrame([new_observation])], ignore_index=True)
